<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_9_%D0%A2%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers stop_words pymorphy2 tqdm nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=931dfaa958e5c0635fa91b2a074ffe90f69181fd4c59df6a0b1678cfe63b1430
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=013bd3de4fe7bd22ad5d26a550f84f1e5141607dd3935e0213b3f1d04df77ae9
  Stored in direct

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
# import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm

# from transformers import pipeline
# from transformers import BertTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import BertForSequenceClassification
from transformers import BertTokenizerFast

from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
import nltk

In [3]:
nltk.download("punkt")
stop_words = set(get_stop_words("ru"))
punctuations = set(punctuation)
lemmatizer = MorphAnalyzer()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'cpu'

In [4]:
tok = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
b_model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

In [5]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels, tokenizer):
        self._labels = labels

        # self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = tokenizer
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [6]:
class BertClassifier(nn.Module):

    def __init__(self, bert_model, dropout=0.5):
        super().__init__()
        # self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.Xrobert = bert_model
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(3, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):

        pooled_output = self.Xrobert(input_ids=x, attention_mask=mask, return_dict=False)[0]
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)

        return final_layer

In [7]:
def preprocess_text(txt, sw, lem, punct):

    txt = str(txt)
    txt = "".join(c for c in txt if c not in punct)
    txt = txt.lower()
    txt = [lem.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip
!unzip -f data.zip

--2023-08-11 12:08:28--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 142.250.153.138, 142.250.153.102, 142.250.153.100, ...
Connecting to drive.google.com (drive.google.com)|142.250.153.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j2f4o8k6se9ovo9v1175qelq7okdhmhr/1691755650000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=f66e882c-45f0-4937-b9d0-e412ce5b2043 [following]
--2023-08-11 12:08:33--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/j2f4o8k6se9ovo9v1175qelq7okdhmhr/1691755650000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=f66e882c-45f0-4937-b9d0-e412ce5b2043
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 108.177.126.132, 

In [ ]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

In [ ]:
df_train.head()

In [ ]:
df_train['class'].value_counts()

In [ ]:
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))
df_val['text'] = df_val['text'].progress_apply(lambda t: preprocess_text(t, stop_words, lemmatizer, punctuations))

In [ ]:
%%time
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train, tok)
valid_dataset = TwitterDataset(df_val['text'], y_val, tok)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [ ]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

In [ ]:
model = BertClassifier(b_model).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.Xrobert.classifier.parameters(), lr=0.0016)  # неполное обучение

In [ ]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.Xrobert.classifier.parameters()]))

In [ ]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0
    model.train()

    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')